In [1]:
import os
import numpy as np
from pathlib import Path
from bioio import BioImage
from glob import glob
import matplotlib.pyplot as plt
import bioio_czi
import imagej
from skimage import io
from skimage.filters import meijering
import segmentation as sg
from skimage.transform import resize
from skimage import util
from skimage import filters
import time
import shutil
from natsort import natsorted
import re
import os
import warnings
warnings.filterwarnings('ignore')
os.environ['JAVA_HOME']=r'C:\Users\zeiss\Downloads\fiji-win64\Fiji.app\java\win64\zulu8.60.0.21-ca-fx-jdk8.0.322-win_x64\jre\bin'

In [2]:
import imagej

# initialize ImageJ2 with Fiji plugins
ij = imagej.init('sc.fiji:fiji')
print(f"ImageJ2 version: {ij.getVersion()}")

ImageJ2 version: 2.15.0/1.54f


In [3]:
folders = Path(r'F:\rebinning_trial').glob('*/')
folders = natsorted(list(folders))
folders 

[WindowsPath('G:/Data/Zeiss/Chip_growth_experiments/growth_tracking/rebinning trial/20241212_DAOM197198_0.50mM_Myr_G53A-C_Syringeless')]

In [ ]:
for folder in folders:
    days =natsorted(list(folder.glob('*.czi')))
    for day in days:
        #create savefolder
        savefolder_string =  re.sub(r'_[dD]ay\d+', '', day.stem)
        savefolder = Path('rebinning_test',savefolder_string)
        savefolder.mkdir(parents=True,exist_ok=True)
        #save processed files
        filename = f'{Path(day.stem)}.tif'
        #if Path(savefolder,'Masks',filename).exists()!=True:
            #try:
            #print current file
        print(filename)
        
        # Get an AICSImage object
        img = BioImage(day, reconstruct_mosaic=False)
        dask_data = img.get_image_dask_data('MZYX', S=2)
        print(dask_data.shape)

        #Rebin and save as tiff
        rebinned_data = np.zeros(np.array(dask_data.shape)//(1,1,2,2))
        for M in range(dask_data.shape[0]):
        
            # load and minimal project
            chunk_loaded = dask_data[M,:,...]
            chunk_resized = resize(chunk_loaded,np.array(chunk_loaded.shape)//(1,2,2))
            rebinned_data[M] = chunk_resized

        io.imsave(Path(savefolder,day.with_suffix('.tif')),rebinned_data)
            
        #savemetadata
        ome = img.metadata
        xml_bytes = ET.tostring(ome, encoding='utf-8')
        xml_text = xml_bytes.decode('utf-8')
        savefile_metadata = Path(savefolder,day.with_suffix('.xml'))
        print(savefile_metadata)
        with open(savefile_metadata, 'w', encoding='utf-8') as f:
            f.write(xml_text)

G53A_D197198_MSR_pMyr0.5mM_Day0.tif
(90, 13, 1104, 1600)
G:\Data\Zeiss\Chip_growth_experiments\growth_tracking\rebinning trial\20241212_DAOM197198_0.50mM_Myr_G53A-C_Syringeless\G53A_D197198_MSR_pMyr0.5mM_Day0.xml
G53A_D197198_MSR_pMyr0.5mM_Day4.tif
(90, 13, 1104, 1600)
